In [25]:
from docx import Document
import pandas as pd
import re
from collections import defaultdict
import urllib
from os import listdir
from os.path import isfile, join, abspath

In [37]:

datafiles = [f for f in listdir('./data/') if isfile(join('./data/', f))]
columns = ['Дата', 'Адрес', 'Оксид углерода', 'Оксид азота', 'Диоксид азота', 'Диоксид серы', 'Озон', 'Взвешенные частицы PM10', 'Взвешенные частицы PM2.5']

In [48]:
def addDate(fn):
    df = pd.DataFrame(columns=columns)
    with open(abspath('data/'+fn), 'rb') as f:
        pollution_report = Document(f)

        adr_cnt = 0
        for i, paragraph in enumerate(pollution_report.paragraphs):
            row_df = dict.fromkeys(columns)
            adress = re.findall("адресу: (.*?)\\(станции", paragraph.text)
            if adress:
                row_df['Адрес'] = adress[0]
                row_df['Дата']  = fn.split('.')[0]
                for j in range(1, len(pollution_report.tables[adr_cnt].rows)):
                    row = pollution_report.tables[adr_cnt].rows[j]
                    row_text = []
                    for cell in row.cells:
                        row_text.append(cell.text)
                    row_df[row_text[0]] = row_text[1]
                df = df.append(row_df, ignore_index=True)
                adr_cnt += 1
                print(df)
    return df

In [49]:
df = pd.DataFrame(columns=columns)

for file in datafiles:
    df = df.append(addDate(df, file))

TypeError: addDate() takes 1 positional argument but 2 were given

In [47]:
df.head()

AttributeError: 'NoneType' object has no attribute 'head'

In [5]:
#https://api.mapbox.com/geocoding/v5/mapbox.places/.json?bbox=30.0874,59.8003,30.556,60.07&access_token=
bbox = 'bbox=30.0874,59.8003,30.556,60.07'
api_auth_key = ''
base_url = 'https://api.mapbox.com'
geocoding_url = '/geocoding/v5/{endpoint}/{search_text}.json'